# Clasificación de Retornos de Radar con Gaussian Naive Bayes

## Introducción

En este notebook se implementa un flujo completo de procesamiento y análisis para la clasificación de señales de radar utilizando el modelo **Gaussian Naive Bayes**. El conjunto de datos proviene del dataset *Ionosphere Radar Returns*, el cual contiene mediciones de 34 características extraídas de retornos de radar. Cada muestra está etiquetada como perteneciente a una de dos clases:

- **'g'** (good): retorno de radar que indica la presencia de una estructura ionosférica coherente.
- **'b'** (bad): retorno de radar que indica la ausencia de dicha estructura.

El modelo Gaussian Naive Bayes asume que las características siguen distribuciones normales condicionadas a la clase. A lo largo del notebook, se exploran estas suposiciones, se preparan los datos, se visualizan las distribuciones, se aplican pruebas de normalidad y se implementa el clasificador desde cero.

## Objetivos

- **Cargar y explorar** el conjunto de datos de retornos de radar.
- **Analizar estadísticamente** las características del dataset.
- **Visualizar** la distribución de cada característica y compararla con una distribución normal teórica.
- **Aplicar pruebas de normalidad** para evaluar la validez de la suposición de Gaussian Naive Bayes.
- **Implementar** el cálculo manual de probabilidades a priori, likelihoods y probabilidades posteriores.
- **Evaluar** el desempeño del clasificador mediante métricas estándar (accuracy, precision, recall, F1-score).
- **Visualizar** la distribución de probabilidades posteriores y analizar la confianza del modelo.

Este ejercicio no solo permite consolidar los conceptos teóricos detrás de Naive Bayes, sino también comprender la importancia de la exploración previa de los datos y de la validación de las suposiciones del modelo.

---


## Descripción del Dataset

El dataset utilizado en este análisis es el **Ionosphere Radar Returns Dataset**, disponible en el repositorio del curso. Este conjunto de datos contiene medidas obtenidas a partir de señales de radar dirigidas hacia la ionosfera.

Cada muestra del dataset corresponde a un pulso de radar, para el cual se han registrado 34 características numéricas que describen propiedades físicas de la señal recibida, como amplitudes y fases de los retornos de radar en diferentes momentos y frecuencias.

El objetivo es predecir si la señal de radar corresponde a un retorno **"bueno"** (estructura coherente en la ionosfera) o **"malo"** (retorno disperso sin estructura), etiquetado en la columna `label` con las siguientes clases:

- `'g'` → *good*: el radar detecta una estructura coherente.
- `'b'` → *bad*: el radar no detecta una estructura coherente.

### Estructura del dataset

- **Número de muestras**: 351
- **Número de características**: 34 (todas numéricas y continuas)
- **Variable objetivo**: `label` (clase binaria: `'g'` o `'b'`)

### Columnas

- `feature1`, `feature3`, `feature4`, ..., `feature34`: características numéricas extraídas de la señal de radar.
- `label`: variable objetivo que indica la clase de la muestra.

### Contexto del problema

El análisis de estos retornos de radar es importante en aplicaciones relacionadas con la **detección de irregularidades en la ionosfera**, que afectan la propagación de señales de radio. Un clasificador automático como Gaussian Naive Bayes puede ser útil para automatizar la clasificación de retornos en sistemas de radar de vigilancia.

---


## Paso 1: Cargar y explorar el dataset

**Instrucciones:**
- Descarga el dataset desde el repositorio de GitHub del curso. El archivo se encuentra en `datasets/ionosphere_radar_returns/original data.csv`.
- Carga el dataset utilizando pandas.
- Muestra las primeras filas (`df.head()`) del dataset.
- Imprime la cantidad total de filas y columnas del dataset.

In [ ]:
# tu código aquí

## Paso 2: División del dataset en entrenamiento y prueba

**Instrucciones:**
- Construye la función `load_and_split_dataset` para cargar el dataset y dividirlo en conjuntos de entrenamiento y prueba.
- Asegúrate de que la variable objetivo sea `'label'`.
- Realiza la división estratificada para mantener la proporción de clases.
- Asigna los conjuntos resultantes a las variables `X_train`, `X_test`, `y_train` y `y_test`.
- Asigna una proporción del 20% a los datos de prueba y 80% a los datos de entrenamiento.

In [ ]:
# tu código aquí

---

En esta asignación se utilizará el modelo Gaussian Naive Bayes, el cual emplea la función de densidad normal para calcular la verosimilitud (likelihood) de cada característica (feature) dado cada clase. Para ello, será necesario calcular la media y la desviación estándar de cada columna del conjunto de entrenamiento. Estos parámetros permitirán estimar la probabilidad de observar los valores de cada muestra bajo la hipótesis de normalidad, lo que es necesario para el cálculo de las probabilidades posteriores y la clasificación.

---

## Paso 3: Estadística descriptiva y visualización de los datos

**Instrucciones:**
- Calcula estadísticas descriptivas (mínimo, máximo, media y desviación estándar) para cada característica del conjunto de entrenamiento.
- Muestra los resultados en una tabla utilizando la función `'label'`

In [ ]:
# tu código aquí

---

Antes de proceder con la clasificación, es fundamental analizar la distribución de los datos. El clasificador Gaussian Naive Bayes asume que las características siguen una distribución normal, pero ¿realmente es así? ¿En qué medida los datos se asemejan a una distribución normal? 

Para responder a estas preguntas, primero visualizaremos los histogramas de frecuencia de cada característica del conjunto de entrenamiento. Esto nos permitirá identificar posibles desviaciones respecto a la normalidad y comprender mejor el comportamiento de los datos antes de aplicar el modelo.

---

## Paso 4: Visualización de histogramas para cada característica

**Instrucciones:**
- Utiliza los datos de entrenamiento (`X_train`) para graficar histogramas de frecuencia para cada característica.
- Organiza los histogramas en una cuadrícula de 2 columnas para facilitar la visualización.
- Asegúrate de etiquetar correctamente los ejes y los títulos de cada subgráfico.
- Añade un título general a la figura.


In [ ]:
# tu código aquí

---

Una manera de aproximar la distribución de los datos observados en un histograma de frecuencias es mediante la **estimación de densidad por kernel** (*Kernel Density Estimation*, KDE). La KDE permite obtener una función suave que aproxima la función de densidad de probabilidad (PDF) de los datos, facilitando la comparación visual con distribuciones teóricas y ayudando a identificar desviaciones respecto a la normalidad o a la forma general de la distribución.

La estimación de densidad por kernel es una técnica **no paramétrica**, es decir, no asume una forma específica para la distribución de los datos. En su lugar, utiliza un **kernel** (por ejemplo, una función gaussiana) para estimar la densidad en cada punto, sumando las contribuciones de todas las muestras en el conjunto de datos. El estimador de densidad se expresa como:

$$
\hat{f}(x) = \frac{1}{n h} \sum_{i=1}^n K\left( \frac{x - x_i}{h} \right)
$$

donde:

- $\hat{f}(x)$ es la densidad estimada en el punto $x$,
- $n$ es el número de muestras,
- $h$ es el parámetro de ancho de banda (controla el grado de suavizado),
- $K(\cdot)$ es la función kernel, típicamente una función gaussiana,
- $x_i$ son las observaciones del conjunto de datos.

Como complemento al uso de la KDE, la representación de un **histograma normalizado** proporciona una estimación discreta de la frecuencia relativa de los valores observados. Mientras que el histograma muestra la frecuencia de aparición de los datos en intervalos discretos, la KDE ofrece una estimación continua y suavizada de la densidad de probabilidad subyacente.

La visualización conjunta del histograma normalizado y de la KDE facilita el análisis exploratorio de las variables. En particular, permite:

- detectar **asimetrías** o sesgos en la distribución,
- identificar posibles **modas múltiples** (multimodalidad),
- evaluar la **normalidad** de los datos.

Este análisis es especialmente relevante cuando se utilizan modelos que suponen normalidad en las variables predictoras, como el **clasificador Gaussian Naive Bayes**. Si las distribuciones de los datos se alejan considerablemente de la normalidad, puede ser necesario aplicar técnicas de transformación o reconsiderar el uso de modelos basados en dicha suposición.

---


## Paso 5: Comparación entre la PDF estimada y la PDF normal teórica

**Instrucciones:**

- Para cada característica del conjunto de entrenamiento (`X_train`), genera una visualización que incluya en un mismo subgráfico:
    - El **histograma normalizado** de la característica, representando la frecuencia relativa de los valores observados.
    - La **PDF estimada** de los datos mediante **KDE** (*Kernel Density Estimation*), proporcionando una curva suave de la densidad.
- Organiza los subgráficos en una cuadrícula con **2 columnas** para facilitar la comparación visual entre las diferentes características.
- Etiqueta de forma adecuada los **ejes** y asigna un **título descriptivo** a cada subgráfico.
- Incluye una **leyenda** en cada subgráfico para distinguir claramente el histograma y la KDE.
- Añade un **título general** a la figura que resuma el propósito de la visualización.


In [ ]:
# tu código aquí

---

Si todo se ha realizado correctamente, al finalizar este paso se podrá visualizar la función de densidad de probabilidad (PDF) de cada característica mediante la KDE superpuesta al histograma normalizado.

El modelo Gaussian Naive Bayes asume que cada variable sigue una distribución normal. Comparar la KDE con la PDF normal teórica permite evaluar visualmente qué tan bien se aproxima la distribución real de los datos a esta suposición.

Aunque Gaussian Naive Bayes se aplicará en cualquier caso, este análisis ayuda a interpretar mejor los resultados del modelo y a comprender el comportamiento de cada variable dentro del marco de esta suposición de normalidad.

---

## Paso 5: Comparación visual entre la PDF estimada y la PDF normal teórica

**Instrucciones:**
- Para cada característica del conjunto de entrenamiento (`X_train`), grafica en el mismo subgráfico:
    - El histograma de la característica (normalizado a densidad).
    - La PDF estimada de los datos usando KDE (Kernel Density Estimation).
    - La PDF normal teórica calculada con la media y desviación estándar de los datos.
- Organiza los subgráficos en una cuadrícula de 2 columnas.
- Añade leyendas, títulos y etiquetas a cada subgráfico para identificar cada curva.
- Incluye un título general para la figura.

In [ ]:
# tu código aquí

---

¿Cómo se puede determinar si una distribución de datos es **normal**?  
Una forma visual es comparar la **KDE** de los datos con la **PDF normal teórica** calculada a partir de la media y la desviación estándar de la muestra. Sin embargo, esta comparación es cualitativa y subjetiva.

La manera más aceptada y rigurosa para evaluar la normalidad es mediante el uso de **pruebas estadísticas formales de normalidad**. Estas pruebas generan un valor estadístico y un **p-valor** que permite cuantificar la evidencia contra la hipótesis nula de que los datos provienen de una distribución normal. Entre las pruebas más utilizadas se encuentran:

- **D'Agostino y Pearson** (`normaltest` de `scipy.stats`):  
  Evalúa la normalidad combinando medidas de **asimetría** y **curtosis**. Es adecuada para muestras de tamaño moderado o grande.

- **Shapiro-Wilk** (`shapiro` de `scipy.stats`):  
  Una de las pruebas más potentes para detectar desviaciones de la normalidad en muestras pequeñas o medianas.

- **Anderson-Darling** (`anderson` de `scipy.stats`):  
  Basada en la función de distribución acumulada. Proporciona umbrales críticos específicos para diferentes niveles de significancia.

Cada prueba tiene sus propias ventajas y sensibilidad a diferentes tipos de desviaciones (asimetría, colas pesadas, multimodalidad, etc.). Por ello, es recomendable utilizar varias pruebas en conjunto y **tomar una decisión combinada**, por ejemplo, mediante un criterio de "votación" (si la mayoría de las pruebas rechazan la normalidad, se concluye que la distribución no es normal).

Este enfoque complementa la inspección visual con un análisis cuantitativo más robusto sobre la distribución de los datos.

---



## Paso 7: Pruebas de normalidad para cada característica

**Instrucciones:**
- Realiza pruebas de normalidad sobre cada característica del conjunto de entrenamiento (`X_train`) utilizando los siguientes tests:
    - D'Agostino y Pearson (`normaltest`)
    - Shapiro-Wilk (`shapiro`)
    - Anderson-Darling (`anderson`)
- Para cada característica, reporta si pasa o no cada prueba (p > 0.05 para normaltest y shapiro; estadístico < valor crítico al 5% para anderson).
- Realiza una "votación mayoritaria" para decidir si la característica puede considerarse normal (al menos 2 de 3 pruebas deben indicar normalidad).
- Muestra los resultados en una tabla con una columna para cada test y una columna final indicando el resultado de la votación mayoritaria.


In [ ]:
# tu código aquí

---

Ahora vamos a utilizar el **clasificador Gaussian Naive Bayes**. Para ello, es necesario aplicar el **Teorema de Bayes**, que proporciona un marco probabilístico para calcular la probabilidad de que una muestra pertenezca a una determinada clase, dado un conjunto de características observadas.

### Modelo matemático

El Teorema de Bayes se expresa de la siguiente manera:

$$
P(C_k \mid \mathbf{x}) = \frac{P(\mathbf{x} \mid C_k) \, P(C_k)}{P(\mathbf{x})}
$$

donde:

- $ P(C_k \mid \mathbf{x}) $ es la **probabilidad posterior**, es decir, la probabilidad de que la muestra pertenezca a la clase $ C_k $ dado el vector de características $ \mathbf{x} $.
- $ P(\mathbf{x} \mid C_k) $ es la **verosimilitud** (*likelihood*), es decir, la probabilidad de observar $ \mathbf{x} $ si la muestra pertenece a la clase $ C_k $.
- $ P(C_k) $ es la **probabilidad a priori** (*prior*), es decir, la probabilidad de que una muestra pertenezca a la clase $ C_k $ antes de observar los datos.
- $ P(\mathbf{x}) $ es la **probabilidad marginal**, que actúa como constante de normalización y es igual para todas las clases; por tanto, no es necesaria para la comparación entre clases.

En la práctica, lo que se necesita para clasificar una muestra es calcular $ P(C_k \mid \mathbf{x}) $ para cada clase y asignar la muestra a la clase con la mayor probabilidad posterior.

### Componentes del cálculo

El cálculo requiere los siguientes componentes:

- **Prior** $ P(C_k) $: se calcula a partir de la frecuencia relativa de cada clase en los datos de entrenamiento:

$$
P(C_k) = \frac{\text{número de muestras de la clase } C_k}{\text{número total de muestras}}
$$

- **Likelihood** $ P(\mathbf{x} \mid C_k) $: en el caso de Gaussian Naive Bayes, se asume que cada característica sigue una **distribución normal** condicional a la clase. Por lo tanto, la likelihood de cada característica se calcula usando la función de densidad de probabilidad normal:

$$
P(x_i \mid C_k) = \frac{1}{\sigma_{k,i} \sqrt{2\pi}} \exp\left( -\frac{(x_i - \mu_{k,i})^2}{2\sigma_{k,i}^2} \right)
$$

donde $ \mu_{k,i} $ y $ \sigma_{k,i}^2 $ son la media y la varianza de la característica $ x_i $ en la clase $ C_k $.

Dado que Naive Bayes asume **independencia condicional** entre características, la likelihood conjunta se calcula como el producto de las likelihoods individuales:

$$
P(\mathbf{x} \mid C_k) = \prod_{i=1}^n P(x_i \mid C_k)
$$

### Resumen

En resumen, para clasificar una muestra $ \mathbf{x} $, el modelo calcula:

$$
P(C_k \mid \mathbf{x}) \propto P(\mathbf{x} \mid C_k) \, P(C_k)
$$

para cada clase $ C_k $, y selecciona la clase con el valor más alto.  
Este es el principio básico detrás del funcionamiento de **Gaussian Naive Bayes**.

---


## Paso 8: Cálculo de probabilidades a priori (priors)

**Instrucciones:**
- Calcula la probabilidad a priori de cada clase (`'g'` y `'b'`) utilizando los datos de entrenamiento (`y_train`).

In [ ]:
# Tu código aquí

---

## Paso 9: Selección de una muestra aleatoria para probar el algoritmo

**Instrucciones:**
- Selecciona una muestra aleatoria del conjunto de prueba (`X_test`) para utilizarla como ejemplo en el cálculo manual de probabilidades y clasificación.
- Muestra los valores de las características de la muestra seleccionada y su etiqueta real (`y_test`).

In [ ]:
# Tu código aquí

---
Luego de calcular las probabilidades a priori (*priors*) y seleccionar una muestra, el siguiente paso es calcular la **verosimilitud** (*likelihood*) de cada característica de la muestra bajo cada clase. En el contexto de Gaussian Naive Bayes, esto significa estimar la probabilidad de observar el valor de cada característica suponiendo que sigue una distribución normal, cuyos parámetros (media y desviación estándar) se estiman a partir de los datos de entrenamiento de cada clase.

**Concepto:**  
La verosimilitud mide cuán probable es observar un valor específico de una característica, dado que la muestra pertenece a una clase particular. En Gaussian Naive Bayes, se asume que cada característica es independiente y sigue una distribución normal condicionada a la clase.

**Modelo matemático:**  
Para una característica $x_i$ de la muestra y una clase $C_k$, la verosimilitud se calcula como:

$$
P(x_i \mid C_k) = \frac{1}{\sigma_{k,i} \sqrt{2\pi}} \exp\left( -\frac{(x_i - \mu_{k,i})^2}{2\sigma_{k,i}^2} \right)
$$

donde:
- $x_i$ es el valor observado de la característica $i$ en la muestra,
- $\mu_{k,i}$ es la media de la característica $i$ en la clase $C_k$,
- $\sigma_{k,i}$ es la desviación estándar de la característica $i$ en la clase $C_k$.


---

## Paso 10: Cálculo del likelihood para cada característica

**Instrucciones:**
- Para la muestra aleatoria seleccionada, calcula la verosimilitud (*likelihood*) de cada característica bajo la hipótesis de cada clase (`'g'` y `'b'`).
- Utiliza la función de densidad normal, empleando la media y desviación estándar estimadas en el conjunto de entrenamiento para cada clase.
- Presenta los resultados en una tabla que muestre, para cada característica, el valor observado y el likelihood bajo cada clase.

In [ ]:
# tu codigo aquí

---

Es posible graficar las **campanas de Gauss** (funciones de densidad normal) correspondientes a cada clase utilizando la media y desviación estándar estimadas en el conjunto de entrenamiento. Esto permite visualizar, para cada característica, cómo el valor observado de la muestra seleccionada se posiciona sobre la curva de cada clase. 

Así, se ilustra de manera intuitiva y gráfica de dónde se obtienen los valores de *likelihood* que intervienen en el cálculo de la probabilidad posterior en Gaussian Naive Bayes. Esta visualización facilita la comprensión del proceso de clasificación y la interpretación probabilística del modelo.



---


## Paso 11: Visualización de la función de densidad normal y el valor observado

**Instrucciones:**
- Para cada característica de la muestra seleccionada, grafica la función de densidad normal (PDF) correspondiente a cada clase, utilizando la media y desviación estándar estimadas en el conjunto de entrenamiento.
- Marca sobre cada curva el valor observado de la muestra seleccionada.
- Asegúrate de que en cada gráfico se visualice claramente la PDF de cada clase y la posición del valor observado, para ilustrar de dónde proviene el valor de *likelihood* utilizado en el cálculo de la probabilidad posterior.


In [ ]:
# tu codigo aquí

---
En Gaussian Naive Bayes, la **probabilidad posterior** de una clase $C_k$ dado un vector de características $\mathbf{x}$ se calcula usando el Teorema de Bayes:

$$
P(C_k \mid \mathbf{x}) \propto P(\mathbf{x} \mid C_k) \cdot P(C_k)
$$

Donde:
- $P(C_k)$ es la probabilidad a priori de la clase $C_k$.
- $P(\mathbf{x} \mid C_k)$ es la verosimilitud, que bajo la hipótesis de independencia condicional se calcula como el producto de las probabilidades individuales de cada característica.

Sin embargo, multiplicar muchas probabilidades pequeñas puede causar **underflow numérico**. Para evitarlo, se trabaja en el **espacio logarítmico**:

$$
\log P(C_k \mid \mathbf{x}) \propto \log P(C_k) + \sum_{i=1}^n \log P(x_i \mid C_k)
$$

Esto transforma los productos en sumas y mejora la estabilidad numérica.  
Finalmente, para obtener probabilidades interpretables, se **normalizan** los valores exponenciando los log-posteriores y dividiendo por la suma total:

$$
P(C_k \mid \mathbf{x}) = \frac{\exp(\log P(C_k \mid \mathbf{x}))}{\sum_j \exp(\log P(C_j \mid \mathbf{x}))}
$$

La clase asignada será aquella con mayor probabilidad posterior. Este procedimiento permite comparar la confianza del modelo en su predicción y analizar la contribución de cada clase al resultado final.

---

## Paso 12: Cálculo y normalización de los log-posteriores

**Instrucciones:**

- Para la muestra seleccionada, calcula la **probabilidad posterior logarítmica** (*log-posterior*) para cada clase, utilizando los parámetros estimados en el conjunto de entrenamiento.
- Realiza este cálculo para cada clase (`'g'` y `'b'`).
- Normaliza las probabilidades obtenidas para que sumen 1, de modo que representen una distribución de probabilidad válida.
- Muestra los valores de log-posterior y las probabilidades normalizadas para cada clase.
- Interpreta a qué clase sería asignada la muestra según el mayor valor de probabilidad posterior.
- Compara con la etiqueta real de la muestra seleccionada.

In [ ]:
# tu codigo aquí

---

El proceso detallado anteriormente (cálculo de probabilidades a priori, verosimilitud, log-posterior y asignación de clase) debe repetirse para **todas las muestras** del conjunto de prueba para evaluar el desempeño global del clasificador. Solo así es posible obtener métricas objetivas como accuracy, precision, recall y F1-score, y analizar la capacidad real del algoritmo para generalizar sobre datos no vistos. Este enfoque sistemático permite comparar el rendimiento del modelo en todo el dataset y no solo en un caso puntual.

---

## Paso 13: Predicción y evaluación del clasificador Naive Bayes

**Instrucciones:**
- Utiliza el modelo Gaussian Naive Bayes implementado para predecir las clases de todas las muestras del conjunto de prueba (`X_test`).

In [ ]:
# tu codigo aquí

---
### ¿Qué es una matriz de confusión?

La **matriz de confusión** es una herramienta fundamental para evaluar el desempeño de un modelo de clasificación. Es una tabla que compara las predicciones del modelo con las etiquetas reales, permitiendo visualizar y cuantificar los aciertos y errores cometidos.

En un problema de clasificación binaria, la matriz de confusión tiene la siguiente estructura:

|                | Predicho: Positivo | Predicho: Negativo |
|----------------|-------------------|--------------------|
| **Real: Positivo** | Verdaderos Positivos (TP) | Falsos Negativos (FN) |
| **Real: Negativo** | Falsos Positivos (FP)     | Verdaderos Negativos (TN) |

- **Verdaderos Positivos (TP):** Casos en los que el modelo predice correctamente la clase positiva.
- **Verdaderos Negativos (TN):** Casos en los que el modelo predice correctamente la clase negativa.
- **Falsos Positivos (FP):** Casos en los que el modelo predice la clase positiva cuando en realidad es negativa (error tipo I).
- **Falsos Negativos (FN):** Casos en los que el modelo predice la clase negativa cuando en realidad es positiva (error tipo II).

La matriz de confusión permite calcular métricas como **accuracy**, **precision**, **recall** y **F1-score**, y analizar qué tipo de errores son más frecuentes, lo que es clave para interpretar y mejorar el modelo.

---

## Paso 14: Visualización detallada de la matriz de confusión

**Instrucciones:**
- Calcula la matriz de confusión utilizando las predicciones del clasificador Gaussian Naive Bayes sobre el conjunto de prueba (`X_test`).
- Visualiza la matriz de confusión como un mapa de calor (heatmap) utilizando `matplotlib`.
- Añade etiquetas claras para los ejes (clase real vs. clase predicha) y un título descriptivo.
- Interpreta visualmente los tipos de errores cometidos por el modelo (falsos positivos y falsos negativos).

In [ ]:
# tu codigo aquí

---
### Métricas de desempeño: Accuracy, Precision, Recall y F1-score

Para evaluar el rendimiento de un clasificador, se utilizan métricas que resumen la calidad de las predicciones en función de los verdaderos positivos (TP), verdaderos negativos (TN), falsos positivos (FP) y falsos negativos (FN):

- **Accuracy (Exactitud):**  
    Mide la proporción de predicciones correctas sobre el total de muestras.
    
    $$
    \text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}
    $$

- **Precision (Precisión):**  
    Indica la proporción de verdaderos positivos entre todas las muestras que el modelo predijo como positivas.
    
    $$
    \text{Precision} = \frac{TP}{TP + FP}
    $$

- **Recall (Sensibilidad o Exhaustividad):**  
    Mide la proporción de verdaderos positivos que fueron correctamente identificados por el modelo.
    
    $$
    \text{Recall} = \frac{TP}{TP + FN}
    $$

- **F1-score:**  
    Es la media armónica entre precisión y recall, proporcionando un balance entre ambas métricas.
    
    $$
    \text{F1-score} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
    $$

Estas métricas permiten analizar no solo cuántas predicciones fueron correctas, sino también la calidad de las predicciones positivas y la capacidad del modelo para detectar correctamente la clase de interés.


---

## Paso 15: Cálculo de métricas de desempeño del clasificador

**Instrucciones:**
- Calcula las métricas de desempeño estándar para el clasificador Gaussian Naive Bayes utilizando las predicciones sobre el conjunto de prueba (`X_test`).
- Reporta las siguientes métricas:
    - **Accuracy** (exactitud)
    - **Precision** (precisión)
    - **Recall** (sensibilidad)
    - **F1-score**
- Presenta los resultados en una tabla clara y bien etiquetada.

In [ ]:
# tu codigo aquí

---
## Interpretación del histograma de probabilidades posteriores

El **histograma de las probabilidades posteriores** para la clase `'g'` permite analizar la **confianza** del modelo en sus predicciones. Cada barra del histograma representa la cantidad de muestras del conjunto de prueba para las cuales la probabilidad posterior de pertenecer a la clase `'g'` cae dentro de un determinado rango.

- Si la mayoría de las probabilidades están **cerca de 0 o 1**, significa que el modelo toma decisiones con **alta confianza**: asigna a la mayoría de las muestras una probabilidad muy baja o muy alta de pertenecer a la clase `'g'`.
- Si muchas probabilidades están **cerca de 0.5** (el umbral de decisión), el modelo está **menos seguro** y las muestras son **ambiguas**: no hay suficiente evidencia para favorecer claramente una clase sobre la otra.
- Un modelo confiado y bien ajustado mostrará una **distribución bimodal**, con la mayoría de las probabilidades agrupadas cerca de los extremos (0 y 1), y pocas muestras cerca del 0.5.

Observar la forma de este histograma ayuda a diagnosticar si el modelo es **decisivo** o **indeciso** en sus predicciones, y puede orientar sobre la necesidad de mejorar el modelo o recolectar más datos para las regiones ambiguas.

---

## Paso 16: Distribución de probabilidades posteriores para la clase 'g'

**Instrucciones:**
- Calcula las probabilidades posteriores de la clase `'g'` para todas las muestras del conjunto de prueba (`X_test`) utilizando el clasificador Gaussian Naive Bayes implementado.
- Grafica un histograma de estas probabilidades posteriores para visualizar su distribución.
- Añade una línea vertical roja en el valor 0.5 para indicar el umbral de decisión del clasificador.

In [ ]:
# tu codigo aquí

## Preguntas para responder

1. ¿Cuál es el objetivo del modelo **Gaussian Naive Bayes**? ¿Por qué es adecuado o no para este conjunto de datos?

In [ ]:
# Respuesta a la pregunta 1

2. ¿Qué hipótesis hace **Gaussian Naive Bayes** sobre la distribución de las características? ¿Qué limitaciones podría tener esta suposición?

In [ ]:
# Respuesta a la pregunta 2

3. ¿Qué observaste al comparar las **KDE** de las características con las **PDF normales teóricas**? ¿Alguna característica se ajusta bien a una distribución normal? ¿Cuál(es) no?

In [ ]:
# Respuesta a la pregunta 3

4. ¿Por qué es importante calcular las **probabilidades a priori** (*priors*)? ¿Qué representa esta probabilidad en el contexto de este dataset? Demuestre cual sería el resultado de la clasificación si no se considerara el *prior* en el cálculo de la probabilidad posterior.

In [ ]:
# Respuesta a la pregunta 4

5. Explica cómo se calcula el **likelihood** de una muestra en **Gaussian Naive Bayes**. ¿Por qué se utiliza el logaritmo de los *likelihoods* en la práctica?

In [ ]:
# Respuesta a la pregunta 5

6. ¿Es necesario realizar **normalización de datos** como parte del preprocesamiento en este dataset? Justifique su respuesta.

In [ ]:
# Respuesta a la pregunta 6

7. Al realizar las **pruebas de normalidad** para cada característica, ¿qué conclusiones puede sacar sobre la validez de la hipótesis de normalidad en este conjunto de datos?

In [ ]:
# Respuesta a la pregunta 7

8. Analice la **matriz de confusión** obtenida. ¿Qué tipo de errores comete el modelo (falsos positivos o falsos negativos)? ¿Qué implicaciones tienen estos errores en el contexto de este problema?

In [ ]:
# Respuesta a la pregunta 8

9. ¿Qué información aporta el histograma de las **probabilidades posteriores** para la clase `'g'`? ¿Qué puede concluir sobre la **confianza** del modelo en sus predicciones?

In [ ]:
# Respuesta a la pregunta 9

10. Basándose en las métricas obtenidas (**accuracy**, **precision**, **recall**, **F1-score**), ¿considera que el modelo es adecuado para este problema? Compare los resultados obtenidos con los de un clasificador **KNN** (de `scikit-learn`) con `k=3` y el resto de parámetros por defecto. Justifique su respuesta.

In [ ]:
# Respuesta a la pregunta 10

## Rúbrica de Evaluación

El trabajo práctico será evaluado con un máximo de **200 puntos**, distribuidos de la siguiente manera:

### 1. Ejecución y desarrollo del notebook (160 puntos)

Cada paso del notebook se evaluará según su correcta implementación, completitud y claridad del código (incluyendo visualizaciones cuando corresponda).  
Cada paso vale **10 puntos**.

| Paso | Descripción                                               | Puntos |
|------|-----------------------------------------------------------|--------|
| 1    | Carga y exploración del dataset                           | 10     |
| 2    | División del dataset en entrenamiento y prueba            | 10     |
| 3    | Estadística descriptiva y visualización de los datos      | 10     |
| 4    | Visualización de histogramas para cada característica     | 10     |
| 5    | Comparación entre PDF estimada y PDF normal teórica       | 10     |
| 6    | Evaluación visual de las distribuciones                   | 10     |
| 7    | Pruebas de normalidad para cada característica            | 10     |
| 8    | Cálculo de probabilidades a priori                        | 10     |
| 9    | Selección y visualización de una muestra aleatoria        | 10     |
| 10   | Cálculo y visualización de la verosimilitud para la muestra| 10     |
| 11   | Visualización de la función de densidad normal            | 10     |
| 12   | Cálculo y normalización de los log-posteriores            | 10     |
| 13   | Predicción y evaluación del clasificador Naive Bayes      | 10     |
| 14   | Visualización detallada de la matriz de confusión         | 10     |
| 15   | Cálculo de métricas de desempeño del clasificador         | 10     |
| 16   | Distribución de probabilidades posteriores para la clase 'g'| 10    |

### 2. Preguntas de reflexión y análisis (40 puntos)

Cada pregunta será evaluada según la **calidad del razonamiento**, **claridad de la explicación**, y en los casos de comparación, la **correcta interpretación de los resultados**.

| Pregunta | Descripción resumida                                   | Puntos |
|----------|-------------------------------------------------------|--------|
| 1        | Objetivo de Gaussian Naive Bayes                      | 2      |
| 2        | Hipótesis de Gaussian Naive Bayes                     | 2      |
| 3        | Comparación KDE vs PDF normal                         | 4      |
| 4        | Importancia del prior y su impacto                    | 8      |
| 5        | Cálculo del likelihood y uso de log                   | 4      |
| 6        | Necesidad de normalización de datos                   | 3      |
| 7        | Conclusiones sobre pruebas de normalidad              | 3      |
| 8        | Análisis de la matriz de confusión                    | 4      |
| 9        | Interpretación del histograma de probabilidades       | 2      |
| 10       | Comparación Naive Bayes vs KNN (con justificación)    | 8      |

### Total: **200 puntos**

---

**Notas:**

- Para obtener la máxima puntuación, es importante que tanto el **código** como las **respuestas** sean completas, bien justificadas y correctamente argumentadas.
- Las respuestas deben apoyarse en los resultados obtenidos y en las visualizaciones cuando sea pertinente.
- Las preguntas que implican **comparación de modelos** o **análisis conceptual profundo** tienen mayor peso en la evaluación.
- Las comparaciones entre modelos y análisis de resultados se evaluarán especialmente en términos de **razonamiento crítico** y no simplemente en repetir valores numéricos.

---
